In [ ]:
import time
from io import BytesIO

import pandas as pd
import win32clipboard
from PIL import Image
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("disable-infobars")  # Deshabilitar infobars
options.add_argument("--disable-notifications")  # Deshabilitar notifications

options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

In [ ]:
cService = webdriver.ChromeService(executable_path='./drivers/chromedriver.exe')
browser = webdriver.Chrome(service=cService)

In [ ]:
browser.get("https://web.whatsapp.com/")
time.sleep(5)

In [ ]:
def paste_content(driver, el, content):
    driver.execute_script(
        f'''
        const text = `{content}`;
        const dataTransfer = new DataTransfer();
        dataTransfer.setData('text', text);
        const event = new ClipboardEvent('paste', {{clipboardData: dataTransfer, bubbles: true}});
        arguments[0].dispatchEvent(event)''',
        el)

In [ ]:
def send_to_clipboard(clip_type, image_data):
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(clip_type, image_data)
    win32clipboard.CloseClipboard()

In [ ]:
## Load image to Bytes
filepath = "images/image_promo.jpg"
image = Image.open(filepath)

output = BytesIO()
image.convert("RGB").save(output, "BMP")
image_data = output.getvalue()[14:]
output.close()

In [ ]:
message_promotion = """
Quieres mejorar tu señal de internet  y tener 1,000 megas a solo 69.90 soles?
Somos WOW!!! Internet 100% Fibra Óptica!  🖥️🏡
Se parte de la comunidad que disfruta de la mayor velocidad al menor precio !! 🛜✨ 🏃‍♂️🏃‍♀️
1️⃣ Deseo obtener 1000 Mbps + DGO TV (GRATIS por 12 meses ) + PARAMOUNT CHANEL por 3 meses  a un super precio de s/ 69.90🏃‍♂️🏃‍♀️

2️⃣ Quiero  mas información 🛜✨
📢 WOW DAME LUZ📢
"""

In [ ]:
## Bucle
# Lee el archivo CSV en un DataFrame
df = pd.read_csv("data/masivo_numeros.csv", dtype={'NUMERO': str, 'ESTADO_ENVIADO': str})

# Itera sobre cada fila del DataFrame
for index, row in df.iterrows():
    if row['ESTADO_ENVIADO'] == "1" or row['ESTADO_ENVIADO'] == "2":
        continue
    try:
        print(str(row['NUMERO']))
        time.sleep(0.2)
        browser.find_element(By.XPATH, "//*[@id='app']/div/div[3]/div/div[3]/header/header/div/span/div/div[1]/button/span").click()
        time.sleep(0.2)
        browser.find_element(By.XPATH, "//*[@id='app']/div/div[3]/div/div[2]/div[1]/span/div/span/div/div[1]/div[2]/div[2]/div/div/p").send_keys(str(row['NUMERO']))

        time.sleep(1)
        if len(browser.find_elements(By.XPATH, "//*[@id='app']/div/div[3]/div/div[2]/div[1]/span/div/span/div/div[2]/div[2]")) == 0:
            browser.find_element(By.XPATH, "//*[@id='app']/div/div[3]/div/div[2]/div[1]/span/div/span/div/header/div/div[1]/div/span").click()
            df.at[index, 'ESTADO_ENVIADO'] = 2
            continue
        browser.find_element(By.XPATH, "//*[@id='app']/div/div[3]/div/div[2]/div[1]/span/div/span/div/div[2]/div[2]").click()

        ## Send Text Promotion
        time.sleep(0.5)
        send_text = browser.find_element(By.XPATH, "//*[@id='main']/footer/div[1]/div/span/div/div[2]/div[1]/div[2]/div[1]/p")
        paste_content(browser, send_text, message_promotion)

        ## Send image
        time.sleep(0.2)
        send_to_clipboard(win32clipboard.CF_DIB, image_data)
        browser.find_element(By.XPATH, "//*[@id='main']/footer/div[1]/div/span/div/div[2]/div[1]/div/div[1]/p").send_keys(Keys.CONTROL + "v")

        ## Send Message
        time.sleep(1)
        browser.find_element(By.XPATH, "//*[@id='app']/div/div[3]/div/div[2]/div[2]/span/div/div/div/div[2]/div/div[2]/div[2]/div/div").click()

        ## Actualizado de csv
        df.at[index, 'ESTADO_ENVIADO'] = 1
    except Exception as e:
        print(f"Error processing row {index}: {e}")

# Guardar el DataFrame actualizado en un nuevo archivo CSV
df.to_csv("data/masivo_numeros.csv", index=False)